In [ ]:
# Importing necessary packages 

## General librariesimport pandas as pd
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import math
import datetime
import time
import seaborn as sns

## Machine Learning libraries
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_regression, f_classif
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from xgboost import XGBClassifier
from datetime import datetime
import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tqdm import tqdm

Sources: 

https://fasttext.cc/docs/en/english-vectors.html
https://github.com/facebookresearch/fastText/blob/a20c0d27cd0ee88a25ea0433b7f03038cd728459/python/doc/examples/train_supervised.py
https://fasttext.cc/docs/en/python-module.html#train_unsupervised-parameters
https://github.com/facebookresearch/fastText/issues/507

https://github.com/justinezth/ML-Text-Classification-with-fastText/blob/master/fasttext_prod_classification.ipynb

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
pip install fasttext

     |████████████████████████████████| 68 kB 6.0 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3148780 sha256=1c5a3b7710c2465f15d858b400a3894de4a4d13f7e4cd08058ecad795c22a10c
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import os
from fasttext import train_supervised

In [ ]:
df_train = pd.read_csv("/content/gdrive/My Drive/243/FakeNewsNet/dataset/clean_merged_data_train.csv").set_index('index')
df_test = pd.read_csv("/content/gdrive/My Drive/243/FakeNewsNet/dataset/clean_merged_data_test.csv").set_index('index')
df_val = pd.read_csv("/content/gdrive/My Drive/243/FakeNewsNet/dataset/clean_merged_data_val.csv").set_index('index')

In [ ]:
df_test.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'title', 'publish_date', 'body', 'tweet_ids', 'fake_news',
       'tweets_count', 'Retweet', 'Like', 'Reply', 'Hour', 'Weekday', 'Month',
       'source_url_billboard', 'source_url_dailymail', 'source_url_elle',
       'source_url_ew', 'source_url_hollywoodreporter', 'source_url_other',
       'source_url_radaronline', 'source_url_today', 'source_url_usatoday',
       'source_url_usmagazine', 'source_url_variety'],
      dtype='object')

In [ ]:
y_train = df_train['fake_news']
y_val = df_val['fake_news']
y_test = df_test['fake_news']

In [ ]:
# function for performance metrics with confusion matrix as argument

def acc(cm):
  return(round((cm[3]+ cm[0])/(cm[2]+cm[3]+cm[1]+cm[0]),3))

def tpr(cm):
  return(round(cm[3]/(cm[2]+cm[3]),3))

def fpr(cm):
  return(round(cm[1]/(cm[0]+cm[1]),3))

def f1(cm):
  return(round(2*cm[3]/(2*cm[3]+cm[2]+cm[1]),3))

### Body

In [ ]:
# only keeping the body of the articles and the labels

df_train_body = df_train[['body','fake_news']]
df_test_body = df_test[['body','fake_news']]
df_val_body = df_val[['body','fake_news']]

##### Train

In [ ]:
# text pre-processing

import os

text = df_train_body['body']

# change to lowercase
text_lowercase = text.str.lower()

# remove punctuation
from string import punctuation
def remove_punctuation(document):
    no_punct = ''.join([character for character in document if character not in punctuation])
    return no_punct
text_no_punct = text_lowercase.apply(remove_punctuation)

# remove digits
def remove_digit(document): 
    no_digit = ''.join([character for character in document if not character.isdigit()])    
    return no_digit
text_no_digit = text_no_punct.apply(remove_digit)

# tokenization (split words)
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
text_tokenized = text_no_digit.apply(word_tokenize)

# remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(document):
    words = [word for word in document if not word in stop_words]
    return words
stop_words = set(stopwords.words('english'))
stop_words.add('XX')
text_no_stop = text_tokenized.apply(remove_stopwords)

# stemming (remove end of words)
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemmer(document):
    stemmed_document = [porter.stem(word) for word in document]
    return stemmed_document
text_stemmed = text_no_stop.apply(stemmer)

# detokenization (merge words)
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_detokenized_body_train = text_stemmed.apply(TreebankWordDetokenizer().detokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
text_detokenized_body_train

index
g_881134        kate middleton want kid know ok talk mental he...
g_866473        maev mcdermott usa today woodi allen clarinet ...
g_914920        cast vanderpump rule choic word costar jame ke...
g_858761        jon hamm definit fan social media made clear i...
g_876437        fan shock former bachelor nick viall announc f...
                                      ...                        
g_2343299673    brad pitt see kid regularli accord new report ...
g_893171        confirm romanc earlier month cosi sight britis...
g_567879122     sure long round rihanna drake last two star ch...
g_884684        celin dion pledg proce show colosseum caesar p...
g_875085        live room bodi coffe tabl would heart ’ gather...
Name: body, Length: 6699, dtype: object

In [ ]:
# exact format for the supervised fastText model: using __label__ + class + text

df_train_body_post = pd.concat([pd.DataFrame(text_detokenized_body_train),df_train_body['fake_news']], axis = 1)
df_train_body_post['fake_news'] = df_train_body_post['fake_news'].astype(str)

In [ ]:
df_train_labeled_body =  '__label__' + df_train_body_post['fake_news'] + ' ' + df_train_body_post['body']
print(df_train_labeled_body)

index
g_881134        __label__0 kate middleton want kid know ok tal...
g_866473        __label__0 maev mcdermott usa today woodi alle...
g_914920        __label__0 cast vanderpump rule choic word cos...
g_858761        __label__0 jon hamm definit fan social media m...
g_876437        __label__0 fan shock former bachelor nick vial...
                                      ...                        
g_2343299673    __label__1 brad pitt see kid regularli accord ...
g_893171        __label__0 confirm romanc earlier month cosi s...
g_567879122     __label__1 sure long round rihanna drake last ...
g_884684        __label__0 celin dion pledg proce show colosse...
g_875085        __label__0 live room bodi coffe tabl would hea...
Length: 6699, dtype: object


##### Test

In [ ]:
# text pre-processing

import os

text = df_test_body['body']

# change to lowercase
text_lowercase = text.str.lower()

# remove punctuation
from string import punctuation
def remove_punctuation(document):
    no_punct = ''.join([character for character in document if character not in punctuation])
    return no_punct
text_no_punct = text_lowercase.apply(remove_punctuation)

# remove digits
def remove_digit(document): 
    no_digit = ''.join([character for character in document if not character.isdigit()])    
    return no_digit
text_no_digit = text_no_punct.apply(remove_digit)

# tokenization (split words)
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
text_tokenized = text_no_digit.apply(word_tokenize)

# remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(document):
    words = [word for word in document if not word in stop_words]
    return words
stop_words = set(stopwords.words('english'))
stop_words.add('XX')
text_no_stop = text_tokenized.apply(remove_stopwords)

# stemming (remove end of words)
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemmer(document):
    stemmed_document = [porter.stem(word) for word in document]
    return stemmed_document
text_stemmed = text_no_stop.apply(stemmer)

# detokenization (merge words)
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_detokenized_body_test = text_stemmed.apply(TreebankWordDetokenizer().detokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# exact format for the supervised fastText model: using __label__ + class + text
df_test_body_post = pd.concat([pd.DataFrame(text_detokenized_body_test),df_test_body['fake_news']], axis = 1)
df_test_body_post['fake_news'] = df_test_body_post['fake_news'].astype(str)

In [ ]:
df_test_labeled_body = df_test_body_post['body']
print(df_test_labeled_body)

index
g_886352    click email friend open new window click share...
g_887153    jaleesa jone special usa today initi sidestep ...
g_851818    canada ’ ‘ freedom convoy ’ trucker whose prot...
g_920809    celebr treat signific other like trash love is...
g_901375    air forc veteran grant die wish see last jedi ...
                                  ...                        
g_900480    look sweet cooki recip latest fashion trend in...
g_900844    perhap anyon els royal famili charlott close r...
g_908022    sinc mycalvin campaign kick last year seen rol...
g_910399    usa today mark sall former glee star plead gui...
g_909782    mani biggest name music miss sunday grammi awa...
Name: body, Length: 2567, dtype: object


##### Validation

In [ ]:
# text pre-processing
import os

text = df_val_body['body']

# change to lowercase
text_lowercase = text.str.lower()

# remove punctuation
from string import punctuation
def remove_punctuation(document):
    no_punct = ''.join([character for character in document if character not in punctuation])
    return no_punct
text_no_punct = text_lowercase.apply(remove_punctuation)

# remove digits
def remove_digit(document): 
    no_digit = ''.join([character for character in document if not character.isdigit()])    
    return no_digit
text_no_digit = text_no_punct.apply(remove_digit)

# tokenization (split words)
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
text_tokenized = text_no_digit.apply(word_tokenize)

# remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(document):
    words = [word for word in document if not word in stop_words]
    return words
stop_words = set(stopwords.words('english'))
stop_words.add('XX')
text_no_stop = text_tokenized.apply(remove_stopwords)

# stemming (remove end of words)
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemmer(document):
    stemmed_document = [porter.stem(word) for word in document]
    return stemmed_document
text_stemmed = text_no_stop.apply(stemmer)

# detokenization (merge words)
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_detokenized_body_val = text_stemmed.apply(TreebankWordDetokenizer().detokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# exact format for the supervised fastText model: using __label__ + class + text

df_val_body_post = pd.concat([pd.DataFrame(text_detokenized_body_val),df_val_body['fake_news']], axis = 1)
df_val_body_post['fake_news'] = df_val_body_post['fake_news'].astype(str)

In [ ]:
df_val_labeled_body = df_val_body_post['body']
print(df_val_labeled_body)

index
p_379           rightli big issu ohio laid critic addit plan a...
g_895866        miley cyru still glad love one lean cant stop ...
g_9109637963    angelina joli file divorc brad pitt twitter fo...
g_857297        us cast came togeth spread fake spoiler season...
g_930760        audrina patridg singer ryan cabrera split back...
                                      ...                        
g_870947        morn aesthetician founder skincar line taught ...
g_891755        spoiler alert read yet watch “ first wife ” ei...
g_908995        mdlla agent give us glimps nuptial like never ...
g_3800163388    wow lot celebr news break sunday night sure ju...
g_844273        author investig caus princ ’ death thursday en...
Name: body, Length: 1002, dtype: object


### Models - body

In [ ]:
#write test and train into txt files

f_train_body = open("fasttext_train_body.txt", "a")
for i in range(len(np.array(df_train_labeled_body))):
    f_train_body.write(np.array(df_train_labeled_body)[i] + "\n")
f_train_body.close()

f_test_body = open("fasttext_test_body.txt", "a")
for i in range(len(np.array(df_test_labeled_body))):
    f_test_body.write(np.array(df_test_labeled_body)[i] + "\n")
f_test_body.close()

f_val_body = open("fasttext_val_body.txt", "a")
for i in range(len(np.array(df_val_labeled_body))):
    f_val_body.write(np.array(df_val_labeled_body)[i] + "\n")
f_val_body.close()

In [ ]:
# training the model with train_supervised

fasttext_body = train_supervised(input="fasttext_train_body.txt", loss = "ns", wordNgrams=1)

In [ ]:
print(fasttext_body.test("fasttext_test_body.txt")) #(n, precision, recall)

(0, nan, nan)


In [ ]:
# predicting the testing set row by row

y_pred_fasttext_body = df_test_labeled_body.apply(lambda x: int(fasttext_body.predict(x)[0][0][-1]))

In [ ]:
# assessing the performance of the model

cm_fasttext_body = confusion_matrix(y_test, y_pred_fasttext_body).ravel()

acc_fasttext_body = acc(cm_fasttext_body)
tpr_fasttext_body = tpr(cm_fasttext_body)
fpr_fasttext_body = fpr(cm_fasttext_body)
f1_fasttext_body = f1(cm_fasttext_body)

print('Accuracy fasttext_body :', acc_fasttext_body)
print('TPR fasttext_body model :', tpr_fasttext_body, '\nFPR fasttext_body model :', fpr_fasttext_body)
print('F1 score fasttext_body model :', f1_fasttext_body)

Accuracy fasttext_body : 0.824
TPR fasttext_body model : 0.564 
FPR fasttext_body model : 0.073
F1 score fasttext_body model : 0.646


In [ ]:
fasttext_body.save_model("/content/gdrive/My Drive/243/FakeNewsNet/fasttext_body.bin")

### Title

In [ ]:
df_train_title = df_train[['title','fake_news']]
df_test_title = df_test[['title','fake_news']]
df_val_title = df_val[['title','fake_news']]

##### Train

In [ ]:
# text pre-processing

import os

text = df_train_title['title']

# change to lowercase
text_lowercase = text.str.lower()

# remove punctuation
from string import punctuation
def remove_punctuation(document):
    no_punct = ''.join([character for character in document if character not in punctuation])
    return no_punct
text_no_punct = text_lowercase.apply(remove_punctuation)

# remove digits
def remove_digit(document): 
    no_digit = ''.join([character for character in document if not character.isdigit()])    
    return no_digit
text_no_digit = text_no_punct.apply(remove_digit)

# tokenization (split words)
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
text_tokenized = text_no_digit.apply(word_tokenize)

# remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(document):
    words = [word for word in document if not word in stop_words]
    return words
stop_words = set(stopwords.words('english'))
stop_words.add('XX')
text_no_stop = text_tokenized.apply(remove_stopwords)

# stemming (remove end of words)
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemmer(document):
    stemmed_document = [porter.stem(word) for word in document]
    return stemmed_document
text_stemmed = text_no_stop.apply(stemmer)

# detokenization (merge words)
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_detokenized_title_train = text_stemmed.apply(TreebankWordDetokenizer().detokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
text_detokenized_title_train

index
g_881134        kate middleton introduc mental health film pay...
g_866473        topless activist crash woodi allen concert ger...
g_914920        jax taylor brittani cartwright lala kent slam ...
g_858761        jon hamm slam social media visual equival masturb
g_876437        bachelor star nick viall vanessa grimaldi spea...
                                      ...                        
g_2343299673    brad pitt see kid sever time week — insid priv...
g_893171        brooklyn beckham new model girlfriend hana cro...
g_567879122         reason rihanna drake could next beyoncé jay z
g_884684         celin dion donat concert proce vega shoot victim
g_875085            pretti celebr book would look good coffe tabl
Name: title, Length: 6699, dtype: object

In [ ]:
# exact format for the supervised fastText model: using __label__ + class + text

df_train_title_post = pd.concat([pd.DataFrame(text_detokenized_title_train),df_train_title['fake_news']], axis = 1)
df_train_title_post['fake_news'] = df_train_title_post['fake_news'].astype(str)

In [ ]:
df_train_labeled_title =  '__label__' + df_train_title_post['fake_news'] + ' ' + df_train_title_post['title']
print(df_train_labeled_title)

index
g_881134        __label__0 kate middleton introduc mental heal...
g_866473        __label__0 topless activist crash woodi allen ...
g_914920        __label__0 jax taylor brittani cartwright lala...
g_858761        __label__0 jon hamm slam social media visual e...
g_876437        __label__0 bachelor star nick viall vanessa gr...
                                      ...                        
g_2343299673    __label__1 brad pitt see kid sever time week —...
g_893171        __label__0 brooklyn beckham new model girlfrie...
g_567879122     __label__1 reason rihanna drake could next bey...
g_884684        __label__0 celin dion donat concert proce vega...
g_875085        __label__0 pretti celebr book would look good ...
Length: 6699, dtype: object


##### Test

In [ ]:
# text pre-processing

import os

text = df_test_title['title']

# change to lowercase
text_lowercase = text.str.lower()

# remove punctuation
from string import punctuation
def remove_punctuation(document):
    no_punct = ''.join([character for character in document if character not in punctuation])
    return no_punct
text_no_punct = text_lowercase.apply(remove_punctuation)

# remove digits
def remove_digit(document): 
    no_digit = ''.join([character for character in document if not character.isdigit()])    
    return no_digit
text_no_digit = text_no_punct.apply(remove_digit)

# tokenization (split words)
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
text_tokenized = text_no_digit.apply(word_tokenize)

# remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(document):
    words = [word for word in document if not word in stop_words]
    return words
stop_words = set(stopwords.words('english'))
stop_words.add('XX')
text_no_stop = text_tokenized.apply(remove_stopwords)

# stemming (remove end of words)
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemmer(document):
    stemmed_document = [porter.stem(word) for word in document]
    return stemmed_document
text_stemmed = text_no_stop.apply(stemmer)

# detokenization (merge words)
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_detokenized_title_test = text_stemmed.apply(TreebankWordDetokenizer().detokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# exact format for the supervised fastText model: using __label__ + class + text

df_test_title_post = pd.concat([pd.DataFrame(text_detokenized_title_test),df_test_title['fake_news']], axis = 1)
df_test_title_post['fake_news'] = df_test_title_post['fake_news'].astype(str)

In [ ]:
df_test_labeled_title = df_test_title_post['title']
print(df_test_labeled_title)

index
g_886352                makeup artist accus ben affleck grope
g_887153    saturday night live final tackl harvey weinste...
g_851818    famili say jetblu kick flight argument birthda...
g_920809               celebr treat signific other like trash
g_901375        air forc veteran grant die wish see last jedi
                                  ...                        
g_900480       eminem sell “ mom ’ spaghetti ” new popup shop
g_900844           time princess charlott look like royal rel
g_908022    calvin klein cast kardashian jenner sister new...
g_910399    glee actor mark sall plead guilti child porn f...
g_909782       taylor swift ed sheeran drake skip year grammi
Name: title, Length: 2567, dtype: object


##### Validation

In [ ]:
# text pre-processing

import os

text = df_val_title['title']

# change to lowercase
text_lowercase = text.str.lower()

# remove punctuation
from string import punctuation
def remove_punctuation(document):
    no_punct = ''.join([character for character in document if character not in punctuation])
    return no_punct
text_no_punct = text_lowercase.apply(remove_punctuation)

# remove digits
def remove_digit(document): 
    no_digit = ''.join([character for character in document if not character.isdigit()])    
    return no_digit
text_no_digit = text_no_punct.apply(remove_digit)

# tokenization (split words)
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
text_tokenized = text_no_digit.apply(word_tokenize)

# remove stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
def remove_stopwords(document):
    words = [word for word in document if not word in stop_words]
    return words
stop_words = set(stopwords.words('english'))
stop_words.add('XX')
text_no_stop = text_tokenized.apply(remove_stopwords)

# stemming (remove end of words)
from nltk.stem import PorterStemmer
porter = PorterStemmer()
def stemmer(document):
    stemmed_document = [porter.stem(word) for word in document]
    return stemmed_document
text_stemmed = text_no_stop.apply(stemmer)

# detokenization (merge words)
from nltk.tokenize.treebank import TreebankWordDetokenizer
text_detokenized_title_val = text_stemmed.apply(TreebankWordDetokenizer().detokenize)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# exact format for the supervised fastText model: using __label__ + class + text

df_val_title_post = pd.concat([pd.DataFrame(text_detokenized_title_val),df_val_title['fake_news']], axis = 1)
df_val_title_post['fake_news'] = df_val_title_post['fake_news'].astype(str)

In [ ]:
df_val_labeled_title = df_val_title_post['title']
print(df_val_labeled_title)

index
p_379                                    democrat debat cleveland
g_895866        miley cyru celebr th birthday famili lose home...
g_9109637963    brad pitt jennif aniston rekindl romanc despit...
g_857297        us cast share hope fake spoiler season red nos...
g_930760        audrina patridg ryan cabrera attend stagecoach...
                                      ...                        
g_870947                   top aesthetician reveal skincar routin
g_891755        ‘ outland ’ recap clair releg ‘ first wife ’ s...
g_908995        josh flagg share futur wed vow bobbi boyd four...
g_3800163388    fan think kyli jenner look like old kyli remov...
g_844273              hollywood pay tribut legendari artist princ
Name: title, Length: 1002, dtype: object


### Models - title

In [ ]:
#write test and train into txt files

f_train_title = open("fasttext_train_title.txt", "a")
for i in range(len(np.array(df_train_labeled_title))):
    f_train_title.write(np.array(df_train_labeled_title)[i] + "\n")
f_train_title.close()

f_test_title = open("fasttext_test_title.txt", "a")
for i in range(len(np.array(df_test_labeled_title))):
    f_test_title.write(np.array(df_test_labeled_title)[i] + "\n")
f_test_title.close()

f_val_title = open("fasttext_val_title.txt", "a")
for i in range(len(np.array(df_val_labeled_title))):
    f_val_title.write(np.array(df_val_labeled_title)[i] + "\n")
f_val_title.close()

In [ ]:
# # training the model with train_supervised

fasttext_title = train_supervised(input="fasttext_train_title.txt", loss = "ns", wordNgrams=1)

In [ ]:
# predicting the testing set row by row

y_pred_fasttext_title = df_test_labeled_title.apply(lambda x: int(fasttext_title.predict(x)[0][0][-1]))

In [ ]:
# assessing the performance of the model

cm_fasttext_title = confusion_matrix(y_test, y_pred_fasttext_title).ravel()

acc_fasttext_title = acc(cm_fasttext_title)
tpr_fasttext_title = tpr(cm_fasttext_title)
fpr_fasttext_title = fpr(cm_fasttext_title)
f1_fasttext_title = f1(cm_fasttext_title)

print('Accuracy fasttext_title :', acc_fasttext_title)
print('TPR fasttext_title model :', tpr_fasttext_title, '\nFPR fasttext_title model :', fpr_fasttext_title)
print('F1 score fasttext_title model :', f1_fasttext_title)

Accuracy fasttext_title : 0.802
TPR fasttext_title model : 0.586 
FPR fasttext_title model : 0.112
F1 score fasttext_title model : 0.628


In [ ]:
fasttext_title.save_model("/content/gdrive/My Drive/243/FakeNewsNet/fasttext_title.bin")

## Combination with other features

In [ ]:
cols = ['tweets_count', 'Retweet', 'Like', 'Reply', 'Hour', 'Weekday', 'Month',
       'source_url_billboard', 'source_url_dailymail', 'source_url_elle',
       'source_url_ew', 'source_url_hollywoodreporter', 'source_url_other',
       'source_url_radaronline', 'source_url_today', 'source_url_usatoday',
       'source_url_usmagazine', 'source_url_variety']

X_train_combine = df_train.copy()[cols]
X_val_combine = df_val.copy()[cols]
X_test_combine = df_test.copy()[cols]

y_pred_train_fasttext_body = df_train_labeled_body.apply(lambda x: int(fasttext_body.predict(x)[0][0][-1]))
y_pred_train_fasttext_title = df_train_labeled_title.apply(lambda x: int(fasttext_title.predict(x)[0][0][-1]))

y_pred_val_fasttext_body = df_val_labeled_body.apply(lambda x: int(fasttext_body.predict(x)[0][0][-1]))
y_pred_val_fasttext_title = df_val_labeled_title.apply(lambda x: int(fasttext_title.predict(x)[0][0][-1]))

y_pred_test_fasttext_body = df_test_labeled_body.apply(lambda x: int(fasttext_body.predict(x)[0][0][-1]))
y_pred_test_fasttext_title = df_test_labeled_title.apply(lambda x: int(fasttext_title.predict(x)[0][0][-1]))


In [ ]:
X_train_combine['body'] = y_pred_train_fasttext_body
X_train_combine['title'] = y_pred_train_fasttext_title

X_val_combine['body'] = y_pred_val_fasttext_body
X_val_combine['title'] = y_pred_val_fasttext_title

X_test_combine['body'] = y_pred_test_fasttext_body
X_test_combine['title'] = y_pred_test_fasttext_title

### Baseline

In [ ]:
y_train.value_counts()

0    4782
1    1917
Name: fake_news, dtype: int64

In [ ]:
y_test.value_counts()

0    1837
1     730
Name: fake_news, dtype: int64

In [ ]:
acc_baseline = round(y_test.value_counts()[0] / (y_test.value_counts()[0] + y_test.value_counts()[1]),3)
tpr_baseline = 0
fpr_baseline = 0
f1_baseline = 0

print('Accuracy baseline model :', acc_baseline)
print('TPR baseline model :', tpr_baseline, '\nFPR baseline model :', fpr_baseline)
print('F1 score baseline model :', f1_baseline)


Accuracy baseline model : 0.716
TPR baseline model : 0 
FPR baseline model : 0
F1 score baseline model : 0


### Logistic regression

In [ ]:
logreg = LogisticRegression(solver='liblinear', fit_intercept=True)
logreg.fit(X_train_combine,y_train)

y_pred_logreg = logreg.predict(X_test_combine)
cm_logreg = confusion_matrix(y_test,y_pred_logreg).ravel()

acc_logreg = acc(cm_logreg)
tpr_logreg = tpr(cm_logreg)
fpr_logreg = fpr(cm_logreg)
f1_logreg = f1(cm_logreg)

print('Accuracy logreg model :', acc_logreg)
print('TPR logreg model :', tpr_logreg, '\nFPR logreg model :', fpr_logreg)
print('F1 score logreg model :', f1_logreg)

Accuracy logreg model : 0.804
TPR logreg model : 0.595 
FPR logreg model : 0.113
F1 score logreg model : 0.633


### CART

In [ ]:
# Cross validation

grid_values = {'ccp_alpha': np.linspace(0.0, 0.10, 201),
               'min_samples_leaf': [5],
               'min_samples_split': [20],
               'max_depth': [30],
               'random_state': [88]} 
            
dtc = DecisionTreeClassifier()
dtc_cv_acc = GridSearchCV(dtc, param_grid = grid_values, scoring = 'f1', cv=10, verbose=1)
dtc_cv_acc.fit(X_train_combine, y_train) 

print('Grid best parameter ccp_alpha (max. CV f1): ', dtc_cv_acc.best_params_['ccp_alpha'])
print('Grid best score (CV f1): ', dtc_cv_acc.best_score_)

Fitting 10 folds for each of 201 candidates, totalling 2010 fits
Grid best parameter ccp_alpha (max. CV f1):  0.0015
Grid best score (CV f1):  0.8778457218453513


In [ ]:
dtc = DecisionTreeClassifier(min_samples_leaf=5, 
                             ccp_alpha= 0.0015, # found by cross validation
                             criterion = 'gini',
                             min_samples_split = 20,
                             max_depth = 30,
                             random_state = 88)

dtc = dtc.fit(X_train_combine, y_train) #fit the decision tree

In [ ]:
cm_dtc = confusion_matrix(y_test, dtc.predict(X_test_combine)).ravel()

acc_dtc = acc(cm_dtc)
tpr_dtc = tpr(cm_dtc)
fpr_dtc = fpr(cm_dtc)
f1_dtc = f1(cm_dtc)

print('Accuracy Decision Tree Classifier :', acc_dtc)
print('TPR dtc model :', tpr_dtc, '\nFPR dtc model :', fpr_dtc)
print('F1 score dtc model :', f1_dtc)
print(f1_score(y_test,dtc.predict(X_test_combine)))

Accuracy Decision Tree Classifier : 0.841
TPR dtc model : 0.695 
FPR dtc model : 0.1
F1 score dtc model : 0.714
0.7135819845179451


### Random Forest

In [ ]:
# Cross validation for classifier

grid_values = {'max_features': np.linspace(1,18,18, dtype='int32'),
              'min_samples_leaf': [5],
              'n_estimators': [500],
              'random_state': [88]} 


rf = RandomForestClassifier() 
rf_cv = GridSearchCV(rf, param_grid=grid_values, scoring='f1', cv=10)
rf_cv.fit(X_train_combine, y_train)

print('Grid best parameter ccp_alpha (max. CV f1 score): ', rf_cv.best_params_['max_features'])
print('Grid best score (CV f1 score): ', rf_cv.best_score_)

Grid best parameter ccp_alpha (max. CV f1 score):  7
Grid best score (CV f1 score):  0.8923907387461025


In [ ]:
rf = RandomForestClassifier(max_features=7, # found by cross validation
                            min_samples_leaf=5,
                            n_estimators = 500,
                            random_state=88,
                            verbose=2)
rf.fit(X_train_combine, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.0s finished


RandomForestClassifier(max_features=7, min_samples_leaf=5, n_estimators=500,
                       random_state=88, verbose=2)

In [ ]:
cm_rf = confusion_matrix(y_test, rf.predict(X_test_combine)).ravel()

acc_rf = acc(cm_rf)
tpr_rf = tpr(cm_rf)
fpr_rf = fpr(cm_rf)
f1_rf = f1(cm_rf)

print('Accuracy rf :', acc_rf)
print('TPR rf model :', tpr_rf, '\nFPR rf model :', fpr_rf)
print('F1 score rf model :', f1_rf)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Accuracy rf : 0.864
TPR rf model : 0.704 
FPR rf model : 0.073
F1 score rf model : 0.746


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.3s finished


### Gradient Boosting

In [ ]:
l= [[]]

for x in tqdm(np.logspace(9, 16, num=5, base=2, dtype='int32')):
  for y in  np.linspace(2, 18, 5, dtype='int32'):
    gbc = XGBClassifier(n_estimators=x, learning_rate= 0.01, random_state=88, verbose=1,
                                max_leaf_nodes=y) 
    gbc.fit(X_train_combine, y_train)
    l = l + [x,y,f1_score(y_val, gbc.predict(X_val_combine))]

print(l)

100%|██████████| 5/5 [34:05<00:00, 409.18s/it]

[[], 512, 2, 0.726923076923077, 512, 6, 0.726923076923077, 512, 10, 0.726923076923077, 512, 14, 0.726923076923077, 512, 18, 0.726923076923077, 1722, 2, 0.7542857142857142, 1722, 6, 0.7542857142857142, 1722, 10, 0.7542857142857142, 1722, 14, 0.7542857142857142, 1722, 18, 0.7542857142857142, 5792, 2, 0.7703984819734344, 5792, 6, 0.7703984819734344, 5792, 10, 0.7703984819734344, 5792, 14, 0.7703984819734344, 5792, 18, 0.7703984819734344, 19483, 2, 0.7683397683397685, 19483, 6, 0.7683397683397685, 19483, 10, 0.7683397683397685, 19483, 14, 0.7683397683397685, 19483, 18, 0.7683397683397685, 65536, 2, 0.7728155339805827, 65536, 6, 0.7728155339805827, 65536, 10, 0.7728155339805827, 65536, 14, 0.7728155339805827, 65536, 18, 0.7728155339805827]


In [ ]:
gbc = XGBClassifier(n_estimators=6000, learning_rate= 0.01, random_state=88, verbose=1,
                                max_leaf_nodes=14) 
gbc.fit(X_train_combine, y_train)

XGBClassifier(learning_rate=0.01, max_leaf_nodes=11, n_estimators=6000,
              random_state=88, verbose=1)

In [ ]:
cm_gbc = confusion_matrix(y_test, gbc.predict(X_test_combine)).ravel()

acc_gbc = acc(cm_gbc)
tpr_gbc = tpr(cm_gbc)
fpr_gbc = fpr(cm_gbc)
f1_gbc = f1(cm_gbc)

print('Accuracy gbc :', acc_gbc)
print('TPR gbc model :', tpr_gbc, '\nFPR gbc model :', fpr_gbc)
print('F1 score gbc model :', f1_gbc)

Accuracy gbc : 0.878
TPR gbc model : 0.723 
FPR gbc model : 0.061
F1 score gbc model : 0.771
